## Preparation

### Imports

In [7]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
  AutoTokenizer,
  AutoConfig,
  AutoModelForSequenceClassification,
  DataCollatorWithPadding,
  TrainingArguments,
  Trainer
)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig

import evaluate
import torch
import numpy as np


### Loading raw model

In [10]:
model_checkpoint = 'distilbert-base-uncased'

# Define label maps
id2label: {0: "Negative", 1: "Positive"}
label2id: {"Negative": 0, "Positive": 1}

# Generate classification model from checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
  model_checkpoint, num_labels=2
)

c:\Users\Pawel\anaconda3\envs\tensorflow_gpu\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Pawel\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and

### Loading dataset

In [16]:
dataset = load_dataset("imdb")
dataset

Dataset imdb downloaded and prepared to C:/Users/Pawel/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 35.71it/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})